In [1]:
import os, code, sys, time
import numpy as np
import tensorflow as tf
from random import randrange
from numpy import linalg as LA
from sklearn.neighbors import kneighbors_graph
from sklearn.neighbors import NearestNeighbors
import matplotlib
import matplotlib.pyplot as plt
from IPython import display
from data_utils import load_data, normalize, next_minibatch

/home/evan/.pyenv/versions/3.6.3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
'''
Set array backend, seed
'''
rng_seed = 98765
use_gpu = True
np.random.seed(rng_seed)
print('rng_seed = {} '.format(rng_seed))

rng_seed = 98765 


In [3]:
'''
Dataset parameters
'''
num_particles = 16 # defaults 16**3
zX = 0.4
zY = 0.0
#zXb = 0.6
#zYb = 0.2
print('Using redshifts z{}, z{}, with {} particles'.format(zX,zY,num_particles**3))

Using redshifts z0.4, z0.0, with 4096 particles


In [5]:
'''
Load training dataset
'''
X,Y = load_data(zX, zY, num_particles)
X = normalize(X)
Y = normalize(Y)
#Xb, Yb = load_data(zXb, zYb, num_particles)
#Xb = normalize(Xb)
#Yb = normalize(Yb)
print('X.shape: {}'.format(X.shape))

X.shape: (1000, 4096, 6)


In [4]:
'''
Pre-processing: nearest-neighbors and sparse adjacency
'''

'''
SPARSE ADJACENCY MATRIX 
• scikit learn gives a CRS sparse adjacency for example, sparse TF takes COO. Use this for sparse X dense matmul in TF.
• also return adjacency lists and convert alist into index list to be used for generic normalizations (avg, max, etc)
'''

# returns adjacency lists based on NN in coordinate space
def adjacency_list(X_in,k):
    shape_in = X_in.shape
    X_out = np.zeros([shape_in[0],shape_in[1],k],dtype=np.int32)
    for b in range(shape_in[0]):
        X_out[b] = kneighbors_graph(X_in[b,:,:3],k,include_self=True).indices.reshape([shape_in[1],k])
    return X_out

def get_adjacency_list(X_in,k):
    """ search for k nneighbors, and return offsetted indices in adjacency list
    
    Args:
        X_in: input data of shape (mb_size, N, 6)
        k: number of nearest neighbors
    """
    mb_size, N, D = X_in.shape
    X_out = np.zeros([mb_size, N, k],dtype=np.int32)
    for b in range(mb_size):
        # this returns indices of the nn
        graph_idx = kneighbors_graph(X_in[b,:,:3],k,include_self=True).indices.reshape([N,k]) + (N * b)
        X_out[b] = graph_idx
    return X_out

# adjacency list to proper index list for get_item
def alist_to_indexlist(alist):
    """ tiles batch indices to adjacency list for tf.gather
    """
    b, n, k = alist.shape
    #b = alist.shape[0] # batch size
    #n = alist.shape[1] # set size
    #k = alist.shape[2] # number of nn
    id1 = np.reshape(np.arange(b),[b,1])
    id1 = np.tile(id1,n*k).flatten()
    out = np.stack([id1,alist.flatten()],axis=1)
    return out

In [13]:
channels = [6, 8, 16, 32, 16, 8, 3, 8, 16, 32, 16, 8, 3] # 'bi',lowest loss for graph model with 0.01 8
#X_input = tf.placeholder(tf.float32, shape=[None,N_p,k_in], name='X_input')
x10 = X[:10]
layer1 = tf.layers.Dense(8)

AttributeError: module 'tensorflow' has no attribute 'tensor'

AttributeError: 'numpy.ndarray' object has no attribute 'get_shape'